In [ ]:
!pip install gymnasium
!pip install supersuit
!pip install pettingzoo
!pip install stable_baselines3
!pip install sb3_contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.1/517.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.8/846.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
%load_ext autoreload
%autoreload 2

#from pettingzoo.mpe import simple_spread_v3
from pprint import pprint
import math
from enum import Enum
import random
import pygame

import glob
import os
import time
import numpy as np
import time

import torch as th

from supersuit import pettingzoo_env_to_vec_env_v1, concat_vec_envs_v1
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor

from stable_baselines3.common.callbacks import ProgressBarCallback

from sb3_contrib import RecurrentPPO
# from stable_baselines3.common.evaluation import evaluate_policy
# from stable_baselines3 import DQN
# from stable_baselines3.dqn import CnnPolicy, MlpPolicy


import random
import pygame
import numpy as np
from gymnasium.utils import EzPickle

from pettingzoo.mpe._mpe_utils.core import Agent, Landmark, World
from pettingzoo.mpe._mpe_utils.scenario import BaseScenario
from pettingzoo.mpe._mpe_utils.simple_env import SimpleEnv, make_env
from pettingzoo.utils.conversions import parallel_wrapper_fn


from typing import Callable
from pprint import pprint

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
class CustomWorld(World):

	def __init__(self):
		super().__init__()
		self.dt = 0.05
		self.contact_force = 200

	@property
	def entities(self):
		### Modification ###
		# add obstacles to entities
		entities = self.agents + self.landmarks + self.obstacles
		return entities
		####################

	# gather agent action forces
	def apply_action_force(self, p_force):
		# set applied forces
		for i, agent in enumerate(self.agents):
			if agent.movable:
				noise = (
					np.random.randn(*agent.action.u.shape) * agent.u_noise
					if agent.u_noise
					else 0.0
				)
				p_force[i] = agent.action.u + noise
		return p_force

	# integrate physical state
	def integrate_state(self, p_force):
		for i, entity in enumerate(self.entities):
			if not entity.movable:
				continue

			### Modification ###
			# don't let entity go out of the map

			new_entity_pos = entity.state.p_pos + entity.state.p_vel * self.dt
			entity.state.last_p_vel = entity.state.p_vel
			if (new_entity_pos >= -1).all() and (new_entity_pos <= 1).all():
				entity.state.p_pos = new_entity_pos
			####################

			entity.state.p_vel = entity.state.p_vel * (1 - self.damping)
			if p_force[i] is not None:
				entity.state.p_vel += (p_force[i] / entity.mass) * self.dt
			if entity.max_speed is not None:
				speed = np.sqrt(
					np.square(entity.state.p_vel[0]) + np.square(entity.state.p_vel[1])
				)
				if speed > entity.max_speed:
					entity.state.p_vel = (
						entity.state.p_vel
						/ np.sqrt(
							np.square(entity.state.p_vel[0])
							+ np.square(entity.state.p_vel[1])
						)
						* entity.max_speed
					)


class CustomSimpleEnv(SimpleEnv):
	def draw(self):
		# clear screen
		self.screen.fill((255, 255, 255))

		### Modification ###
		# draw border
		pygame.draw.rect(self.screen, (0, 0, 0, 0), rect=pygame.Rect(0, 0, self.width, self.height), width=25)
		####################

		# update bounds to center around agent
		all_poses = [entity.state.p_pos for entity in self.world.entities]
		cam_range = 1

		# update geometry and text positions
		text_line = 0
		for e, entity in enumerate(self.world.entities):
			# geometry
			x, y = entity.state.p_pos
			y *= (
				-1
			)  # this makes the display mimic the old pyglet setup (ie. flips image)
			x = (
				(x / cam_range) * self.width // 2 * 0.9
			)  # the .9 is just to keep entities from appearing "too" out-of-bounds
			y = (y / cam_range) * self.height // 2 * 0.9
			x += self.width // 2
			y += self.height // 2
			pygame.draw.circle(
				self.screen, entity.color * 200, (x, y), entity.size * 350
			)  # 350 is an arbitrary scale factor to get pygame to render similar sizes as pyglet
			pygame.draw.circle(
				self.screen, (0, 0, 0), (x, y), entity.size * 350, 1
			)  # borders
			assert (
				0 < x < self.width and 0 < y < self.height
			), f"Coordinates {(x, y)} are out of bounds."

			# text
			if isinstance(entity, Agent):
				if entity.silent:
					continue
				if np.all(entity.state.c == 0):
					word = "_"
				elif self.continuous_actions:
					word = (
						"[" + ",".join([f"{comm:.2f}" for comm in entity.state.c]) + "]"
					)
				else:
					word = alphabet[np.argmax(entity.state.c)]

				message = entity.name + " sends " + word + "   "
				message_x_pos = self.width * 0.05
				message_y_pos = self.height * 0.95 - (self.height * 0.05 * text_line)
				self.game_font.render_to(
					self.screen, (message_x_pos, message_y_pos), message, (0, 0, 0)
				)
				text_line += 1

	def reset(self, seed=None, options=None):
		if seed is not None:
			self._seed(seed=seed)
		self.scenario.reset_world(self.world, self.np_random)

		self.agents = self.possible_agents[:]
		self.rewards = {name: 0.0 for name in self.agents}
		self._cumulative_rewards = {name: 0.0 for name in self.agents}
		self.terminations = {name: False for name in self.agents}
		self.truncations = {name: False for name in self.agents}
		self.infos = {name: {} for name in self.agents}

		self.agent_selection = self._agent_selector.reset()
		self.steps = 0

		self.current_actions = [None] * self.num_agents

		if False:
			for agent in self.world.agents:
				self._cumulative_rewards[agent.name] = self.scenario.reward(agent, self.world)


class raw_env(CustomSimpleEnv, EzPickle):
	def __init__(
		self,
		N=3,
		O=5,
		local_ratio=0.5,
		max_cycles=25,
		continuous_actions=False,
		render_mode=None,
	):
		EzPickle.__init__(
			self,
			N=N,
			O=O,
			local_ratio=local_ratio,
			max_cycles=max_cycles,
			continuous_actions=continuous_actions,
			render_mode=render_mode,
		)
		assert (
			0.0 <= local_ratio <= 1.0
		), "local_ratio is a proportion. Must be between 0 and 1."
		scenario = PathFindingScenario()
		world = scenario.make_world(N, O)
		CustomSimpleEnv.__init__(
			self,
			scenario=scenario,
			world=world,
			render_mode=render_mode,
			max_cycles=max_cycles,
			continuous_actions=continuous_actions,
			local_ratio=local_ratio,
		)
		self.metadata["name"] = "simple_spread_v3_modified"
		self.scenario = scenario
		self.world = world


custom_env = make_env(raw_env)


class PathFindingScenario(BaseScenario):
	def make_world(self, N=3, O=5, agent_size=0.025, landmark_size = 0.1, obstacle_size = 0.06):
		world = CustomWorld()
		# set any world properties first
		world.dim_c = 2
		world.num_agents = N
		world.num_landmarks = N
		world.num_obstacles = O
		world.collaborative = True

		world.agent_size = agent_size
		world.landmark_size = landmark_size
		world.obstacle_size = obstacle_size

		# add agents
		world.agents = [Agent() for i in range(world.num_agents)]
		for i, agent in enumerate(world.agents):
			agent.name = f"agent_{i}"
			agent.agent_id = i
			agent.collide = True
			agent.silent = True
			agent.size = world.agent_size

		# add landmarks
		world.landmarks = [Landmark() for i in range(world.num_landmarks)]
		for i, landmark in enumerate(world.landmarks):
			landmark.name = f"landmark_{i}"
			landmark.landmark_id = i
			landmark.collide = False
			landmark.movable = False
			landmark.size = world.landmark_size

		# add obstacles
		world.obstacles	= [Landmark() for i in range(world.num_obstacles)]
		for i, obstacle in enumerate(world.obstacles):
			obstacle.name = f"obstacle_{i}"
			obstacle.obstacle_id = i
			obstacle.collide = True
			obstacle.movable = False
			obstacle.size = world.obstacle_size

		return world

	def _place_entity_on_border(self, margin = 0):
		r = random.randint(0, 3)
		minimum = -1 + margin * random.random()
		maximum = 1 - margin * random.random()
		if r == 0:
			x = random.random() * 2 - 1
			y = minimum
		elif r == 1:
			x = random.random() * 2 - 1
			y = maximum
		elif r == 2:
			x = minimum
			y = random.random() * 2 - 1
		else:
			x = maximum
			y = random.random() * 2 - 1
		return np.array([x, y])

	def get_color(self, i, N, axis):
		color_value_base = 0.5
		color_value_max = 0.9
		remaining_range = color_value_max - color_value_base
		step = remaining_range / ((N-1) if N > 1 else 1)

		color_value = color_value_base + step * i
		color = np.zeros((3,))
		color[axis] = color_value
		return color

	def reset_world(self, world, np_random):
		# random properties for agents
		for i, agent in enumerate(world.agents):
			agent.color = self.get_color(i, world.num_agents, 2)

		# random properties for landmarks
		for i, landmark in enumerate(world.landmarks):
			landmark.color = self.get_color(i, world.num_landmarks, 0)

		# random properties for obstacles
		for i, obstacle in enumerate(world.obstacles):
			obstacle.color = np.array([0.30, 0.15, 0.15])

		positions = set()

		# set random initial states
		for i, landmark in enumerate(world.landmarks):
			landmark.state.p_pos = self._place_entity_on_border(landmark.size * 3)  # place landmark on the border
			landmark.state.p_vel = np.zeros(world.dim_p)
			landmark.state.last_p_vel = np.zeros(world.dim_p)
			positions.add(tuple(landmark.state.p_pos))
		for i, agent in enumerate(world.agents):
			agent.state.p_pos = world.landmarks[i].state.p_pos * -1  # place agent in the opposite side of the map as its target
			agent.state.p_vel = np.zeros(world.dim_p)
			agent.state.c = np.zeros(world.dim_c)
			agent.state.last_p_vel = np.zeros(world.dim_p)
		for i, obstacle in enumerate(world.obstacles):
			obstacle.state.p_pos = np_random.uniform(
				-1 + world.obstacle_size * 1.5, +1 - world.obstacle_size * 1.5, world.dim_p)  # place obtsacle randomly inside the map, but not on the border
			obstacle.state.p_vel = np.zeros(world.dim_p)
			positions.add(tuple(obstacle.state.p_pos))
			obstacle.state.last_p_vel = np.zeros(world.dim_p)

		# set collision counters to 0
		world.collision_counter = {}
		for a in world.agents:
			world.collision_counter[a.name] = {
				"agent": 0,
				"obstacle": 0,
				"border": 0
			}
		world.distance_from_target = {}
		for a in world.agents:
			world.distance_from_target[a.name] = 0

	def is_collision(self, agent1, agent2):
		delta_pos = agent1.state.p_pos - agent2.state.p_pos
		dist = np.sqrt(np.sum(np.square(delta_pos)))
		dist_min = agent1.size + agent2.size
		return True if (dist < dist_min) else False

	def get_distance_l2(self, pos1, pos2):
		return np.sqrt(np.sum(np.square(pos1 - pos2)))

	def get_distance_l1(self, pos1, pos2):
		return np.abs(np.sum(np.square(pos1 - pos2)))

	def get_distance_reward_log(self, distance):
		if distance <= 0:
			return -250

		return (np.log(distance) - 1)

	def get_distance_reward_square(self, distance):
		if distance <= 0:
			return 0

		return distance ** 2

	def reward(self, agent, world):
		# Agents are penalized for collisions and for being on the border
		rew = 0
		collisions_agent = 0
		collisions_obstacle = 0
		collisions_border = 0

		reward_per_collision = -5

		if agent.collide:
			for a in world.agents:
				is_collision = (self.is_collision(a, agent) and a != agent)
				if is_collision:
					collisions_agent += 1
					if True:
						rew += reward_per_collision
			for o in world.obstacles:
				is_collision = (self.is_collision(o, agent))
				if is_collision:
					collisions_obstacle += 1
					if True:
						rew += reward_per_collision
		if agent.state.p_pos[0] in (-1, 1) or agent.state.p_pos[1] in (-1 , 1):
			collisions_border += 1
			if False:
				rew += reward_per_collision

		target = world.landmarks[agent.agent_id]
		distance = self.get_distance_l2(agent.state.p_pos, target.state.p_pos) - agent.size - target.size

		r = self.get_distance_reward_log(distance)

		rew -= r

		world.collision_counter[agent.name]["agent"] = collisions_agent
		world.collision_counter[agent.name]["obstacle"] = collisions_obstacle
		world.collision_counter[agent.name]["border"] = collisions_border

		world.distance_from_target[agent.name] = distance

		return rew

	def global_reward(self, world):
		# Reward based on how close the agent is to it's target
		rew = 0
		# TODO: change back
		# for i, agent in enumerate(world.agents):
		#	target = world.landmarks[i]
		#	dist = np.sqrt(np.sum(np.square(agent.state.p_pos - target.state.p_pos)))
		#	rew -= dist
		return rew

	def observation(self, agent, world):
		# get positions of all entities in this agent's reference frame

		entity = world.landmarks[agent.agent_id]
		entity_pos = entity.state.p_pos - agent.state.p_pos

		if self.is_collision(entity, agent):
			entity_pos = np.array([0, 0])

		if False:
			if self.is_collision(entity, agent):
				entity_pos = np.array([0, 0])
			else:
				if entity_pos[0] < 0:
					entity_pos[0] += agent.size
					entity_pos[0] += entity.size
					if entity_pos[0] > 0:
						entity_pos[0] = 0
				elif entity_pos[0] > 0:
					entity_pos[0] -= agent.size
					entity_pos[0] -= entity.size
					if entity_pos[0] < 0:
						entity_pos[0] = 0

				if entity_pos[1] < 0:
					entity_pos[1] += agent.size
					entity_pos[1] += entity.size
					if entity_pos[1] > 0:
						entity_pos[1] = 0
				elif entity_pos[1] > 0:
					entity_pos[1] -= agent.size
					entity_pos[1] -= entity.size
					if entity_pos[1] < 0:
						entity_pos[1] = 0


		# communication of all other agents
		entities = []
		for other in world.agents:
			if other is agent:
				continue

			new_other_pos = other.state.p_pos + other.state.last_p_vel * 0.05
			distance = self.get_distance_l2(other.state.p_pos, agent.state.p_pos)
			distance_from_edges = distance - agent.size - other.size

			# pos
			relative_position = other.state.p_pos - agent.state.p_pos

			relative_position_0 = relative_position[0]
			relative_position_1 = relative_position[1]
			if self.is_collision(other, agent):
				relative_position_0 = 0
				relative_position_1 = 0
			else:
				if relative_position_0 < 0:
					relative_position_0 += agent.size
					relative_position_0 += other.size
					if relative_position_0 > 0:
						relative_position_0 = 0
				elif relative_position_0 > 0:
					relative_position_0 -= agent.size
					relative_position_0 -= other.size
					if relative_position_0 < 0:
						relative_position_0 = 0

				if relative_position_1 < 0:
					relative_position_1 += agent.size
					relative_position_1 += other.size
					if relative_position_1 > 0:
						relative_position_1 = 0
				elif relative_position_1 > 0:
					relative_position_1 -= agent.size
					relative_position_1 -= other.size
					if relative_position_1 < 0:
						relative_position_1 = 0

			relative_position_from_edges = np.array([relative_position_0, relative_position_1])


			# new pos
			relative_position = new_other_pos - agent.state.p_pos

			relative_position_0 = relative_position[0]
			relative_position_1 = relative_position[1]
			if self.is_collision(other, agent):
				relative_position_0 = 0
				relative_position_1 = 0
			else:
				if relative_position_0 < 0:
					relative_position_0 += agent.size
					relative_position_0 += other.size
					if relative_position_0 > 0:
						relative_position_0 = 0
				elif relative_position_0 > 0:
					relative_position_0 -= agent.size
					relative_position_0 -= other.size
					if relative_position_0 < 0:
						relative_position_0 = 0

				if relative_position_1 < 0:
					relative_position_1 += agent.size
					relative_position_1 += other.size
					if relative_position_1 > 0:
						relative_position_1 = 0
				elif relative_position_1 > 0:
					relative_position_1 -= agent.size
					relative_position_1 -= other.size
					if relative_position_1 < 0:
						relative_position_1 = 0

			new_relative_position_from_edges = np.array([relative_position_0, relative_position_1])

			entities.append((relative_position_from_edges, distance_from_edges, new_relative_position_from_edges))

		for obstacle in world.obstacles:
			distance = self.get_distance_l2(obstacle.state.p_pos, agent.state.p_pos)
			distance_from_edges = distance - agent.size - obstacle.size

			relative_position = obstacle.state.p_pos - agent.state.p_pos

			relative_position_0 = relative_position[0]
			relative_position_1 = relative_position[1]
			if self.is_collision(obstacle, agent):
				relative_position_0 = 0
				relative_position_1 = 0
			else:
				if relative_position_0 < 0:
					relative_position_0 += agent.size
					relative_position_0 += obstacle.size
					if relative_position_0 > 0:
						relative_position_0 = 0
				elif relative_position_0 > 0:
					relative_position_0 -= agent.size
					relative_position_0 -= obstacle.size
					if relative_position_0 < 0:
						relative_position_0 = 0

				if relative_position_1 < 0:
					relative_position_1 += agent.size
					relative_position_1 += obstacle.size
					if relative_position_1 > 0:
						relative_position_1 = 0
				elif relative_position_1 > 0:
					relative_position_1 -= agent.size
					relative_position_1 -= obstacle.size
					if relative_position_1 < 0:
						relative_position_1 = 0

			relative_position_from_edges = np.array([relative_position_0, relative_position_1])

			entities.append((relative_position_from_edges, distance_from_edges, relative_position_from_edges))

		entities.sort(key=lambda x: x[1])

		entities_new_pos = [p[2] for p in entities]
		entities = [p[0] for p in entities]

		entities = entities[0]
		entities_new_pos = entities_new_pos[0]

		obs = np.concatenate(
			[entity_pos] + [entities] + [entities_new_pos]
		).astype(float)

		divide_by = np.max(np.abs(obs)).item()
		if divide_by:
			obs /= divide_by

		return obs

In [ ]:
class ModelTrainer:

	def __init__(self, name: str, env_fn: Callable, seed: float, cpu_count: int, n_steps: int, n_epochs: int, progress_bar: bool, batch_size: int, train_settings: dict, eval_settings: dict, learning_rate: float = 0.0001, ent_coef: float = 0.001):
		'''
			Created environment and model
		'''
		print("Creating model for training")

		self.name = name
		self.env_fn = env_fn
		self.seed = seed
		self.cpu_count = cpu_count
		self.n_steps = n_steps
		self.n_epochs = n_epochs
		self.progress_bar = progress_bar
		self.batch_size = batch_size
		self.learning_rate = learning_rate
		self.ent_coef = ent_coef

		self.train_settings = train_settings
		self.eval_settings = eval_settings

		self.env = ModelTrainer.create_env(parallel_wrapper_fn(self.env_fn), self.seed, self.cpu_count, train_settings)

		self.model = ModelTrainer.create_model(self.name, self.env, self.n_steps, self.n_epochs, self.batch_size, self.learning_rate, self.ent_coef)

		self.trained_for = 0

		self.avg_rewards = {}
		self.avg_collisions = {
			"agent": {},
			"obstacle": {},
			"border": {}
		}
		self.avg_distance_from_target = {}
		self.avg_on_target = {}

		self.rewards = {}
		self.collisions = {
			"agent": {},
			"obstacle": {},
			"border": {}
		}
		self.distance_from_target = {}
		self.on_target = {}

	def learn(self, update_count: int = 4, eval_after: int = 2):
		assert update_count >= eval_after, "Not enough updates to reach eval"
		assert (update_count/eval_after).is_integer(), "Must be the same number of updates before every eval"

		if self.trained_for == 0:
			max_reward, avg_reward, avg_agent_collision, avg_obstacle_collision, avg_border_collision, avg_distance_from_target, avg_on_target, rewards, agent_collisions, obstacle_collisions, border_collisions, distance_from_target, on_target = self.eval(None)
			ind = self.trained_for
			self.avg_rewards[ind] = avg_reward
			self.avg_collisions["agent"][ind] = avg_agent_collision
			self.avg_collisions["obstacle"][ind] = avg_obstacle_collision
			self.avg_collisions["border"][ind] = avg_border_collision
			self.avg_distance_from_target[ind] = avg_distance_from_target
			self.avg_on_target[ind] = avg_on_target

			self.rewards[ind] = rewards
			self.collisions["agent"][ind] = agent_collisions
			self.collisions["obstacle"][ind] = obstacle_collisions
			self.collisions["border"][ind] = border_collisions
			self.distance_from_target[ind] = distance_from_target
			self.on_target[ind] = on_target
		else:
			max_reward, avg_reward, avg_agent_collision, avg_obstacle_collision, avg_border_collision, avg_distance_from_target, avg_on_target, rewards, agent_collisions, obstacle_collisions, border_collisions, distance_from_target, on_target = None, None, None, None, None, None, None, None, None, None, None, None, None

		agent_count = self.train_settings["N"]
		env_count = self.train_settings["env_count"]
		timesteps_per_update = agent_count * self.n_steps * env_count
		i = 0
		while i < update_count:
			self.model.learn(total_timesteps=(timesteps_per_update * eval_after), progress_bar=self.progress_bar)
			i += eval_after
			ind = self.trained_for + i
			print(f"Eval at step {ind}")
			max_reward, avg_reward, avg_agent_collision, avg_obstacle_collision, avg_border_collision, avg_distance_from_target, avg_on_target, rewards, agent_collisions, obstacle_collisions, border_collisions, distance_from_target, on_target = self.eval(max_reward)
			self.avg_rewards[ind] = avg_reward
			self.avg_collisions["agent"][ind] = avg_agent_collision
			self.avg_collisions["obstacle"][ind] = avg_obstacle_collision
			self.avg_collisions["border"][ind] = avg_border_collision
			self.avg_distance_from_target[ind] = avg_distance_from_target
			self.avg_on_target[ind] = avg_on_target

			self.rewards[ind] = rewards
			self.collisions["agent"][ind] = agent_collisions
			self.collisions["obstacle"][ind] = obstacle_collisions
			self.collisions["border"][ind] = border_collisions
			self.distance_from_target[ind] = distance_from_target
			self.on_target[ind] = on_target
		self.trained_for += i

	def eval(self, max_reward: float|None) -> tuple[float, float, float, float, float, float]:
		num_games = self.eval_settings["env_count"]
		eval_settings = {k:v for k,v in self.eval_settings.items() if k != "env_count"}
		env = self.env_fn(**eval_settings)

		model = self.model

		rewards = []
		collisions = {
			"agent": [],
			"obstacle": [],
			"border": []
		}
		distance_from_target = []
		on_target = []

		for i in range(num_games):
			env.reset(seed=i)

			rewards.append([])
			collisions["agent"].append([])
			collisions["obstacle"].append([])
			collisions["border"].append([])
			distance_from_target.append([])
			on_target.append([])

			for agent in env.agent_iter():
				obs, reward, termination, truncation, info = env.last()

				if "render_mode" in eval_settings and eval_settings["render_mode"] == "human":
					pygame.event.get()
					time.sleep(0.01 / len(env.possible_agents))

				for agent in env.agents:
					rewards[-1].append(env.rewards[agent])

					col = env.unwrapped.world.collision_counter[agent]
					collisions["agent"][-1].append(col["agent"])
					collisions["obstacle"][-1].append(col["obstacle"])
					collisions["border"][-1].append(col["border"])

					distance_from_target[-1].append(env.unwrapped.world.distance_from_target[agent])
					on_target[-1].append(int(env.unwrapped.world.distance_from_target[agent] <= 0))

				if termination or truncation:
					break
				else:
					act = model.predict(obs, deterministic=True)[0]

				env.step(act)

		env.close()

		avg_reward = np.array(rewards).mean().item()
		rewards = [sum(r) / len(r) for r in rewards]

		agent_collisions = collisions["agent"]
		obstacle_collisions = collisions["obstacle"]
		border_collisions = collisions["border"]
		avg_agent_collision = np.array(agent_collisions).mean().item()
		avg_obstacle_collision = np.array(obstacle_collisions).mean().item()
		avg_border_collision = np.array(border_collisions).mean().item()
		agent_collisions = [sum(r) / len(r) for r in agent_collisions]
		obstacle_collisions = [sum(r) / len(r) for r in obstacle_collisions]
		border_collisions = [sum(r) / len(r) for r in border_collisions]

		avg_distance_from_target = np.array(distance_from_target).mean().item()
		avg_on_target = np.array(on_target).mean().item()
		distance_from_target = [sum(r) / len(r) for r in distance_from_target]
		on_target = [sum(r) / len(r) for r in on_target]

		print(f"Avg reward: {avg_reward}")
		print(f"Avg collisions: agent {avg_agent_collision}, obstacle {avg_obstacle_collision}, border {avg_border_collision}")
		print(f"Avg distance from target: {avg_distance_from_target}")
		print(f"Avg on target: {avg_on_target}")

		if max_reward is None or avg_reward > max_reward:
			max_reward = avg_reward
			save_name = f"{env.unwrapped.metadata.get('name')}_{time.strftime('%Y%m%d-%H%M%S')}"
			print(f'New best reward, saving model with name "{save_name}"')
			model.save(save_name)
		else:
			print(f"reward {avg_reward} is not larger than {max_reward}")

		return max_reward, avg_reward, avg_agent_collision, avg_obstacle_collision, avg_border_collision, avg_distance_from_target, avg_on_target, rewards, agent_collisions, obstacle_collisions, border_collisions, distance_from_target, on_target

	def eval_with_settings(self, settings):
		num_games = settings["env_count"]
		eval_settings = {k:v for k,v in settings.items() if k != "env_count"}
		env = self.env_fn(**eval_settings)

		model = self.model

		for i in range(num_games):
			env.reset(seed=i)

			for agent in env.agent_iter():
				obs, reward, termination, truncation, info = env.last()
				print(agent, "Obs:", obs, "Rew:", reward)

				if "render_mode" in eval_settings and eval_settings["render_mode"] == "human":
					pygame.event.get()
					time.sleep(0.01 / len(env.possible_agents))

				if termination or truncation:
					break
				else:
					act = model.predict(obs, deterministic=True)[0]

				env.step(act)

		env.close()

	@staticmethod
	def eval_with_settings_and_model(settings, model, env_fn):
		num_games = settings["env_count"]
		eval_settings = {k:v for k,v in settings.items() if k != "env_count"}
		env = env_fn(**eval_settings)

		for i in range(num_games):
			env.reset(seed=i)

			for agent in env.agent_iter():
				obs, reward, termination, truncation, info = env.last()
				print(agent, "Obs:", obs, "Rew:", reward)

				if "render_mode" in eval_settings and eval_settings["render_mode"] == "human":
					pygame.event.get()
					time.sleep(0.01 / len(env.possible_agents))

				if termination or truncation:
					break
				else:
					act = model.predict(obs, deterministic=True)[0]

				env.step(act)

		env.close()

	@staticmethod
	def create_env(env_fn, seed: float, cpu_count: int, train_settings: dict):
		env_count = train_settings["env_count"]
		train_settings = {k:v for k,v in train_settings.items() if k != "env_count"}
		env = env_fn(**train_settings)
		env.reset(seed=seed)
		env = pettingzoo_env_to_vec_env_v1(env)
		env = concat_vec_envs_v1(env, env_count, num_cpus=cpu_count, base_class="stable_baselines3")
		print(f"Created env with settings {(train_settings | {'env_count': env_count}).items()}")
		return env

	@staticmethod
	def create_model(name: str, env, n_steps: int, n_epochs: int, batch_size: int, learning_rate: float, ent_coef: float):
		#policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=dict(pi=[512]))
		model = PPO(
			"MlpPolicy",
			env,
			verbose=3,
			n_steps=n_steps,
			learning_rate=0.0001,
			batch_size=batch_size,
			n_epochs=n_epochs,
			ent_coef = ent_coef,
			device="cuda",
			#policy_kwargs=policy_kwargs
		)
		print(f"Created model {name} with n_steps {n_steps}, n_epochs {n_epochs}, batch_size {batch_size}, learning_rate {learning_rate}, ent_coef {ent_coef}")
		return model




In [ ]:
AGENT_COUNT = 16
OBSTACLE_COUNT = 8
MAX_CYCLES = 512
TRAIN_ENV_COUNT = 1
TEST_ENV_COUNT = 4
BASE_ENV_KWARGS = dict(render_mode=None, N=AGENT_COUNT, O=OBSTACLE_COUNT, local_ratio=1/AGENT_COUNT, max_cycles=MAX_CYCLES, continuous_actions=False, env_count=TRAIN_ENV_COUNT)
TRAIN_ENV_KWARGS = BASE_ENV_KWARGS
TEST_ENV_KWARGS = BASE_ENV_KWARGS | {"env_count": TEST_ENV_COUNT}
HUMAN_RENDER_ENV_KWARGS = TEST_ENV_KWARGS | {"env_count": 1, "max_cycles": 128}
ENV_FUNCTION = custom_env

TRAIN_N_STEPS = 1024
TRAIN_N_EPOCHS = 2
TRAIN_BATCH_SIZE = (AGENT_COUNT * TRAIN_ENV_COUNT * TRAIN_N_STEPS) // 2
TRAIN_CPU_COUNT = 4
TRAIN_PROGRESS_BAR = False

# 1

In [ ]:
model_trainer = ModelTrainer(name="4_agent", env_fn=ENV_FUNCTION, seed=None, cpu_count=TRAIN_CPU_COUNT, n_steps=TRAIN_N_STEPS, n_epochs=TRAIN_N_EPOCHS, progress_bar=TRAIN_PROGRESS_BAR, batch_size=TRAIN_BATCH_SIZE, train_settings=TRAIN_ENV_KWARGS, eval_settings=TEST_ENV_KWARGS, learning_rate=0.0001, ent_coef=0.001)

Creating model for training
Created env with settings dict_items([('render_mode', None), ('N', 16), ('O', 8), ('local_ratio', 0.0625), ('max_cycles', 512), ('continuous_actions', False), ('env_count', 1)])
Using cuda device
Created model 4_agent with n_steps 1024, n_epochs 2, batch_size 8192, learning_rate 0.0001, ent_coef 0.001


In [ ]:
model_trainer.learn(update_count=32, eval_after=4)

Avg reward: -0.0045476380657812005
Avg collisions: agent 0.2964497131697791, obstacle 0.0005492493592090809, border 0.04593097766385939
Avg distance from target: 1.9859378629424413
Avg on target: 0.0019528866105211766
New best reward, saving model with name "simple_spread_v3_modified_20231212-214038"
------------------------------
| time/              |       |
|    fps             | 605   |
|    iterations      | 1     |
|    time_elapsed    | 27    |
|    total_timesteps | 16384 |
------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 604           |
|    iterations           | 2             |
|    time_elapsed         | 54            |
|    total_timesteps      | 32768         |
| train/                  |               |
|    approx_kl            | 1.4438738e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.61        

In [ ]:
print(model_trainer.avg_rewards)
print(model_trainer.avg_distance_from_target)
print(model_trainer.avg_on_target)
print(model_trainer.avg_collisions["agent"])
print(model_trainer.avg_collisions["obstacle"])
print(model_trainer.avg_collisions["agent"])

{0: -0.0045476380657812005, 4: 0.5503621155097086, 8: 0.6450077390547027, 12: 0.6673188619946531, 16: 0.6651425311894451, 20: 0.5928804182032804, 24: 0.6021478307872324, 28: 0.6150049586850845, 32: 0.6063391675728885}
{0: 1.9859378629424413, 4: 0.26737092458696793, 8: 0.09124086929176296, 12: 0.08630131509665151, 16: 0.0972542749701073, 20: 0.08938009500106536, 24: 0.10001618740191419, 28: 0.10426230240381726, 32: 0.10995171149010934}
{0: 0.0019528866105211766, 4: 0.5600951269376296, 8: 0.6564426186988893, 12: 0.6801385328939339, 16: 0.6775601122909801, 20: 0.6029346698401075, 24: 0.6118790430855608, 28: 0.6253566306603198, 32: 0.617346744171854}
{0: 0.2964497131697791, 4: 0.011046014890760405, 8: 0.007689491028927133, 12: 0.03533885634077871, 16: 0.009398266813133162, 20: 0.039061546442084705, 24: 0.006713047723666545, 28: 0.011107042597339191, 32: 0.03747482607103625}
{0: 0.0005492493592090809, 4: 0.009001586720371048, 8: 0.0078420602953741, 12: 0.004851702673013548, 16: 0.0085133650

# 2

In [ ]:
model_trainer = ModelTrainer(name="action_and_value", env_fn=ENV_FUNCTION, seed=None, cpu_count=TRAIN_CPU_COUNT, n_steps=TRAIN_N_STEPS, n_epochs=TRAIN_N_EPOCHS, progress_bar=TRAIN_PROGRESS_BAR, batch_size=TRAIN_BATCH_SIZE, train_settings=TRAIN_ENV_KWARGS, eval_settings=TEST_ENV_KWARGS, learning_rate=0.0001, ent_coef=0.001)

Creating model for training
Created env with settings dict_items([('render_mode', None), ('N', 16), ('O', 8), ('local_ratio', 0.0625), ('max_cycles', 512), ('continuous_actions', False), ('env_count', 1)])
Using cuda device
Created model action_and_value with n_steps 1024, n_epochs 2, batch_size 8192, learning_rate 0.0001, ent_coef 0.001


In [ ]:
model_trainer.learn(update_count=32, eval_after=4)

Avg reward: 0.03590755220634628
Avg collisions: agent 0.0671381056999878, obstacle 0.00387525936775296, border 0.011902309898694008
Avg distance from target: 1.2302655905221225
Avg on target: 0.03567450872696204
New best reward, saving model with name "simple_spread_v3_modified_20231212-220329"
------------------------------
| time/              |       |
|    fps             | 618   |
|    iterations      | 1     |
|    time_elapsed    | 26    |
|    total_timesteps | 16384 |
------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 621           |
|    iterations           | 2             |
|    time_elapsed         | 52            |
|    total_timesteps      | 32768         |
| train/                  |               |
|    approx_kl            | 1.1225602e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.61         |
|  

KeyboardInterrupt: ignored

In [ ]:
print(model_trainer.avg_rewards)
print(model_trainer.avg_distance_from_target)
print(model_trainer.avg_on_target)
print(model_trainer.avg_collisions["agent"])
print(model_trainer.avg_collisions["obstacle"])
print(model_trainer.avg_collisions["agent"])

# 3

In [ ]:
model_trainer = ModelTrainer(name="action_and_value", env_fn=ENV_FUNCTION, seed=None, cpu_count=TRAIN_CPU_COUNT, n_steps=TRAIN_N_STEPS, n_epochs=TRAIN_N_EPOCHS, progress_bar=TRAIN_PROGRESS_BAR, batch_size=TRAIN_BATCH_SIZE, train_settings=TRAIN_ENV_KWARGS, eval_settings=TEST_ENV_KWARGS, learning_rate=0.0001, ent_coef=0.001)

In [ ]:
model_trainer.learn(update_count=32, eval_after=4)

In [ ]:
print(model_trainer.avg_rewards)
print(model_trainer.avg_distance_from_target)
print(model_trainer.avg_on_target)
print(model_trainer.avg_collisions["agent"])
print(model_trainer.avg_collisions["obstacle"])
print(model_trainer.avg_collisions["agent"])

# 4

In [ ]:
model_trainer = ModelTrainer(name="action_and_value", env_fn=ENV_FUNCTION, seed=None, cpu_count=TRAIN_CPU_COUNT, n_steps=TRAIN_N_STEPS, n_epochs=TRAIN_N_EPOCHS, progress_bar=TRAIN_PROGRESS_BAR, batch_size=TRAIN_BATCH_SIZE, train_settings=TRAIN_ENV_KWARGS, eval_settings=TEST_ENV_KWARGS, learning_rate=0.0001, ent_coef=0.001)

In [ ]:
model_trainer.learn(update_count=32, eval_after=4)

In [ ]:
print(model_trainer.avg_rewards)
print(model_trainer.avg_distance_from_target)
print(model_trainer.avg_on_target)
print(model_trainer.avg_collisions["agent"])
print(model_trainer.avg_collisions["obstacle"])
print(model_trainer.avg_collisions["agent"])

# 5

In [ ]:
model_trainer = ModelTrainer(name="action_and_value", env_fn=ENV_FUNCTION, seed=None, cpu_count=TRAIN_CPU_COUNT, n_steps=TRAIN_N_STEPS, n_epochs=TRAIN_N_EPOCHS, progress_bar=TRAIN_PROGRESS_BAR, batch_size=TRAIN_BATCH_SIZE, train_settings=TRAIN_ENV_KWARGS, eval_settings=TEST_ENV_KWARGS, learning_rate=0.0001, ent_coef=0.001)

In [ ]:
model_trainer.learn(update_count=32, eval_after=4)

In [ ]:
print(model_trainer.avg_rewards)
print(model_trainer.avg_distance_from_target)
print(model_trainer.avg_on_target)
print(model_trainer.avg_collisions["agent"])
print(model_trainer.avg_collisions["obstacle"])
print(model_trainer.avg_collisions["agent"])

# 6

In [ ]:
model_trainer = ModelTrainer(name="action_and_value", env_fn=ENV_FUNCTION, seed=None, cpu_count=TRAIN_CPU_COUNT, n_steps=TRAIN_N_STEPS, n_epochs=TRAIN_N_EPOCHS, progress_bar=TRAIN_PROGRESS_BAR, batch_size=TRAIN_BATCH_SIZE, train_settings=TRAIN_ENV_KWARGS, eval_settings=TEST_ENV_KWARGS, learning_rate=0.0001, ent_coef=0.001)

In [ ]:
model_trainer.learn(update_count=32, eval_after=4)

In [ ]:
print(model_trainer.avg_rewards)
print(model_trainer.avg_distance_from_target)
print(model_trainer.avg_on_target)
print(model_trainer.avg_collisions["agent"])
print(model_trainer.avg_collisions["obstacle"])
print(model_trainer.avg_collisions["agent"])

In [ ]:
print("asd")

In [ ]:
import time

print(time.time())